In [4]:
import numpy as np
import pandas as pd

import os
import random

import warnings

warnings.filterwarnings(action='ignore')

# path = '/opt/ml/input/data/FE/'
path = '/opt/ml/input/data/'
dat = pd.read_csv(path + 'FE_total2.csv')


In [5]:
dat.drop(['day','year','KnowledgeTag', 'last_answerCode7','last_answerCode8','last_answerCode9','last_answerCode10'], axis = 1, inplace = True)

In [6]:
_train = dat[dat['answerCode'] >= 0]
_test = dat[dat['answerCode'] < 0]

In [7]:
# valid 제작하는 함수 새로 개편.
# 예전 valid 제작 함수는 1분 걸렸는데 0.6초만에 끗~
_train['train_valid'] = 0
_train.loc[_train.drop_duplicates(subset='userID', keep = 'last').index, 'train_valid'] = -1
_valid = _train[_train['train_valid'] == -1]
_train = _train[_train['train_valid'] == 0]

In [8]:
# 모델에 적용하기 전 기본적인 데이터 전처리 부분
## 라벨링, 필요없는 칼럼 제거
_train_value = _train['answerCode']
_train.drop(['Timestamp', 'testId', 'train_valid', 'answerCode'], axis = 1, inplace = True) # ,'category_st_qcut_5'

_valid_value = _valid['answerCode']
_valid.drop(['Timestamp', 'testId', 'train_valid', 'answerCode'], axis = 1, inplace = True) # ,'category_st_qcut_5'

_test.drop(['Timestamp', 'testId', 'answerCode'], axis = 1, inplace = True) # ,'category_st_qcut_5'

In [9]:
# CatBoost에 적용하기 위해선 문자열 데이터로 변환 필요.
# 만약 범주형이 아니고 연속형(정수, 소수) 성질을 살리고 싶은 변수는 여기 drop에 적어두면 됨.(solve_time 같이.)
cat_columns = list(_train.drop(['solve_time','month'],axis=1).columns)

for col in cat_columns:
    _train[col] = _train[col].astype('str')
    _valid[col] = _valid[col].astype('str')
    _test[col] = _test[col].astype('str')

In [18]:
#!pip install optuna
#!pip install catboost
import gc
from sklearn.metrics import accuracy_score, roc_auc_score
from optuna import Trial, visualization
from optuna.samplers import TPESampler
from catboost import CatBoostClassifier
import matplotlib.pyplot as plt


def objective(trial, FEATURE, train, valid, test, train_value, valid_value):
    
    param = {
    "task_type" : "GPU",
    "eval_metric" : 'AUC',
    "devices" : '0',
    'random_state' : trial.suggest_int("random_state", 1, 50000),
    'learning_rate': 0.08,#trial.suggest_loguniform('learning_rate', 0.01, 0.05),
    'bagging_temperature': trial.suggest_loguniform('bagging_temperature', 0.01, 0.1),
    'iterations': 500, #trial.suggest_int("iterations", 500, 2000),
    'max_depth': trial.suggest_int("max_depth", 8, 12),
    'random_strength': trial.suggest_int("random_strength", 1, 50),
    'l2_leaf_reg': trial.suggest_float("l2_leaf_reg",1e-5,9e-5),
    #"colsample_bylevel":trial.suggest_float("colsample_bylevel", 0.4, 1.0),
    'min_child_samples': trial.suggest_int("min_child_samples", 8, 16), 
    'max_bin': trial.suggest_int("max_bin", 220, 320), 
    'od_type': 'Iter',
    "cat_features": list(train.columns),  # list(train[FEATURE])
    }
    
    # param = {
    # "task_type" : "GPU",
    # "eval_metric" : 'AUC',
    # "devices" : '0',
    # "random_state": 1998,
    # 'learning_rate': 0.01,
    # 'bagging_temperature': 0.05,
    # 'iterations': trial.suggest_int("iterations", 4000, 10000),
    # 'max_depth': 11,
    # 'random_strength': 0,
    # 'l2_leaf_reg': 2e-7,
    # 'min_child_samples': 5, 
    # 'max_bin': 100, 
    # 'od_type': 'Iter',
    # "cat_features" : list(_train),
    # }
    
    model = CatBoostClassifier(**param)
    model.fit(train[FEATURE], train_value, early_stopping_rounds=100, verbose=200, eval_set=(valid, valid_value))
    
    print('train score')
    train_pred = model.predict_proba(train[FEATURE])[:,1]
    print(roc_auc_score(train_value, train_pred)) # auc
    print(accuracy_score(train_value, np.where(train_pred >= 0.5, 1, 0))) # acc, 정확도

    print('valid score')
    valid_pred = model.predict_proba(valid[FEATURE])[:,1]
    print(roc_auc_score(valid_value, valid_pred)) # auc
    print(accuracy_score(valid_value, np.where(valid_pred >= 0.5, 1, 0))) # acc, 정확도

    gc.collect()

    # feature_importance = model.feature_importances_
    # sorted_idx = np.argsort(feature_importance)
    # fig = plt.figure(figsize=(12, 6))
    # plt.barh(range(len(sorted_idx)), feature_importance[sorted_idx], align="center")
    # plt.yticks(range(len(sorted_idx)), np.array(_valid.columns)[sorted_idx])
    # plt.title("Feature Importance")


    return roc_auc_score(valid_value, valid_pred)

***optuna 적용***

In [19]:
import optuna
sampler = optuna.samplers.TPESampler(seed=2022128)
study = optuna.create_study(
    study_name = 'cat_parameter_opt',
    direction = 'maximize',
    sampler = sampler,
)
study.optimize(lambda trial : objective(trial, cat_columns, _train, _valid, _test, _train_value, _valid_value), n_trials=30)

[I 2022-12-08 08:56:07,476] A new study created in memory with name: cat_parameter_opt
Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.7984627	best: 0.7984627 (0)	total: 304ms	remaining: 10m 8s
200:	test: 0.8299349	best: 0.8299349 (200)	total: 46.8s	remaining: 6m 59s
400:	test: 0.8360879	best: 0.8360879 (400)	total: 1m 33s	remaining: 6m 12s
600:	test: 0.8384539	best: 0.8384539 (600)	total: 2m 19s	remaining: 5m 24s
800:	test: 0.8400306	best: 0.8400346 (799)	total: 3m 5s	remaining: 4m 36s
1000:	test: 0.8411784	best: 0.8412201 (993)	total: 3m 51s	remaining: 3m 51s
1200:	test: 0.8421048	best: 0.8421048 (1200)	total: 4m 37s	remaining: 3m 4s
1400:	test: 0.8428372	best: 0.8428372 (1400)	total: 5m 23s	remaining: 2m 18s
1600:	test: 0.8434511	best: 0.8434709 (1598)	total: 6m 7s	remaining: 1m 31s
1800:	test: 0.8438002	best: 0.8438196 (1789)	total: 6m 51s	remaining: 45.5s
1999:	test: 0.8439954	best: 0.8440595 (1973)	total: 7m 37s	remaining: 0us
bestTest = 0.8440595269
bestIteration = 1973
Shrink model to first 1974 iterations.
train score
0.9005854238158655
0.835389439963407
valid score
0.8440595696351237
0.76552002149

[I 2022-12-08 09:04:35,819] Trial 0 finished with value: 0.8440595696351237 and parameters: {'bagging_temperature': 0.061053157936408836, 'random_strength': 15, 'l2_leaf_reg': 1.178392730173686e-05, 'min_child_samples': 13, 'max_bin': 235}. Best is trial 0 with value: 0.8440595696351237.
Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.7937209	best: 0.7937209 (0)	total: 275ms	remaining: 9m 9s
200:	test: 0.8306880	best: 0.8306880 (200)	total: 48.8s	remaining: 7m 16s
400:	test: 0.8367829	best: 0.8367829 (400)	total: 1m 36s	remaining: 6m 24s
600:	test: 0.8392512	best: 0.8392512 (600)	total: 2m 23s	remaining: 5m 33s
800:	test: 0.8406407	best: 0.8406407 (800)	total: 3m 9s	remaining: 4m 43s
1000:	test: 0.8416978	best: 0.8416978 (1000)	total: 3m 54s	remaining: 3m 54s
1200:	test: 0.8423153	best: 0.8423293 (1198)	total: 4m 39s	remaining: 3m 5s
1400:	test: 0.8427735	best: 0.8427939 (1383)	total: 5m 24s	remaining: 2m 18s
1600:	test: 0.8430421	best: 0.8430449 (1599)	total: 6m 7s	remaining: 1m 31s
1800:	test: 0.8431858	best: 0.8432328 (1748)	total: 6m 52s	remaining: 45.6s
bestTest = 0.8432328105
bestIteration = 1748
Shrink model to first 1749 iterations.
train score
0.8931618272480527
0.8306314755447062
valid score
0.8432328761222393
0.7659231389411448


[I 2022-12-08 09:12:24,896] Trial 1 finished with value: 0.8432328761222393 and parameters: {'bagging_temperature': 0.0403316977206044, 'random_strength': 5, 'l2_leaf_reg': 1.3394113749299223e-05, 'min_child_samples': 13, 'max_bin': 291}. Best is trial 0 with value: 0.8440595696351237.
Default metric period is 5 because AUC is/are not implemented for GPU


...

[I 2022-12-08 10:00:11,387] Trial 7 finished with value: 0.8440124456475033 and parameters: {'bagging_temperature': 0.1390486881838301, 'random_strength': 8, 'l2_leaf_reg': 4.1720653680766065e-06, 'min_child_samples': 14, 'max_bin': 264}. Best is trial 6 with value: 0.844602651909631.
Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.7984627	best: 0.7984627 (0)	total: 292ms	remaining: 9m 43s
200:	test: 0.8299570	best: 0.8299570 (200)	total: 47.5s	remaining: 7m 4s
400:	test: 0.8362352	best: 0.8362352 (400)	total: 1m 34s	remaining: 6m 16s
600:	test: 0.8386339	best: 0.8386426 (599)	total: 2m 20s	remaining: 5m 27s
800:	test: 0.8402472	best: 0.8402489 (799)	total: 3m 6s	remaining: 4m 39s
1000:	test: 0.8414336	best: 0.8414510 (992)	total: 3m 53s	remaining: 3m 52s
1200:	test: 0.8423315	best: 0.8423315 (1200)	total: 4m 39s	remaining: 3m 5s
1400:	test: 0.8430843	best: 0.8431055 (1393)	total: 5m 23s	remaining: 2m 18s
1600:	test: 0.8435135	best: 0.8435250 (1597)	total: 6m 10s	remaining: 1m 32s
1800:	test: 0.8438468	best: 0.8438468 (1800)	total: 6m 55s	remaining: 45.9s
bestTest = 0.8441063762
bestIteration = 1894
Shrink model to first 1895 iterations.
train score
0.898532247507412
0.8346632180722442
valid score
0.8441061876903615
0.7657887664606289


[I 2022-12-08 10:08:39,989] Trial 8 finished with value: 0.8441061876903615 and parameters: {'bagging_temperature': 0.03781548050307085, 'random_strength': 10, 'l2_leaf_reg': 3.412662253382621e-06, 'min_child_samples': 16, 'max_bin': 246}. Best is trial 6 with value: 0.844602651909631.
Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.7984627	best: 0.7984627 (0)	total: 304ms	remaining: 10m 7s
200:	test: 0.8300590	best: 0.8300590 (200)	total: 48.6s	remaining: 7m 15s
400:	test: 0.8361781	best: 0.8361788 (398)	total: 1m 34s	remaining: 6m 17s
600:	test: 0.8386232	best: 0.8386284 (598)	total: 2m 21s	remaining: 5m 28s
800:	test: 0.8402292	best: 0.8402292 (800)	total: 3m 7s	remaining: 4m 40s
1000:	test: 0.8415130	best: 0.8415130 (1000)	total: 3m 54s	remaining: 3m 53s
1200:	test: 0.8424156	best: 0.8424595 (1198)	total: 4m 39s	remaining: 3m 5s
1400:	test: 0.8431230	best: 0.8431230 (1400)	total: 5m 25s	remaining: 2m 19s
1600:	test: 0.8436931	best: 0.8436931 (1600)	total: 6m 9s	remaining: 1m 32s
1800:	test: 0.8440896	best: 0.8440896 (1800)	total: 6m 53s	remaining: 45.7s
1999:	test: 0.8442539	best: 0.8442847 (1977)	total: 7m 40s	remaining: 0us
bestTest = 0.8442846537
bestIteration = 1977
Shrink model to first 1978 iterations.
train score
0.8997606821588309
0.8347223799430934
valid score
0.8442846372692487
0.767535608

[I 2022-12-08 10:17:07,999] Trial 9 finished with value: 0.8442846372692487 and parameters: {'bagging_temperature': 0.1054336291901036, 'random_strength': 8, 'l2_leaf_reg': 9.24081625413373e-06, 'min_child_samples': 12, 'max_bin': 238}. Best is trial 6 with value: 0.844602651909631.


***feature 중요도***

In [ ]:
arange(0, train.shape[0]-1)

***optuna에서 나왔던 값들로 param 돌려보기***

In [ ]:

#best param
param = {
    "task_type" : "GPU",
    "eval_metric" : 'AUC',
    "devices" : '0',
    'learning_rate': 0.01,#trial.suggest_loguniform('learning_rate', 0.01, 0.05),
    'iterations': 6000, #trial.suggest_int("iterations", 500, 2000),
    'od_type': 'Iter',
    "cat_features" : cat_columns,
    'random_state': 5753, 
    'bagging_temperature': 0.013328266865025943, 
    'max_depth': 11, 
    'random_strength': 16, 
    'l2_leaf_reg': 6.530922242678823e-05, 
    'min_child_samples': 72, 
    'max_bin': 229
    }


model = CatBoostClassifier(**param)
model.fit(_train, _train_value, early_stopping_rounds=1000, verbose=10, eval_set=(_valid, _valid_value))


In [ ]:
import matplotlib.pyplot as plt

feature_importance = model.feature_importances_
sorted_idx = np.argsort(feature_importance)
fig = plt.figure(figsize=(12, 6))
plt.barh(range(len(sorted_idx)), feature_importance[sorted_idx], align="center")
plt.yticks(range(len(sorted_idx)), np.array(_valid.columns)[sorted_idx])
plt.title("Feature Importance")

In [ ]:
# submission 제출하기 위한 코드

_test_pred = cat_model.predict_proba(_test)[:,1]
_test['prediction'] = _test_pred
submission = _test['prediction'].reset_index(drop = True).reset_index()
submission.rename(columns = {'index':'id'}, inplace = True)
submission.to_csv('../output/cat_v1.csv', index = False)